In [ ]:
import os
import numpy as np
import keras

from keras import layers
from sklearn.model_selection import train_test_split
from src import data_preprocessing_2 as preproc

In [2]:
csv_path = r'data\raw\v2\pynq_1_data.csv'
save_directory = r'data\processed'
X, y = preproc.preprocess_csv(csv_path, 0, save_dir=save_directory)['puf_response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Processed data saved to data\processed\lfsr_seed_processed_2.csv
Processed data saved to data\processed\puf_response_processed_2.csv


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=(130,)),
        layers.Dense(256, activation="relu", name="hidden_layer_1",),
        layers.Dense(256, activation="relu", name="hidden_layer_2"),
        layers.Dense(128, activation="sigmoid", name="output_layer"),
    ]
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.summary()

In [5]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0626 - loss: 0.2266 - val_accuracy: 0.0875 - val_loss: 0.0043
Epoch 2/50
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1060 - loss: 0.0025 - val_accuracy: 0.0849 - val_loss: 4.6666e-04
Epoch 3/50
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1865 - loss: 3.3242e-04 - val_accuracy: 0.2621 - val_loss: 1.2575e-04
Epoch 4/50
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2015 - loss: 9.5183e-05 - val_accuracy: 0.0819 - val_loss: 4.2924e-05
Epoch 5/50
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1078 - loss: 3.3682e-05 - val_accuracy: 0.0901 - val_loss: 1.6384e-05
Epoch 6/50
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0932 - loss: 1.2911e-05 - val_accuracy: 0.0823 - val_loss: 6.4484e-06
Epoch 7/50
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0915 - loss: 5.1145e-06 - val_accuracy: 0.0851 - val_loss: 2.5823e-06
Epoch 8/50
1660/1660 ━━━━━━━━━━━━━━━━━━━━ 3s 1

In [6]:
model.save("models/puf_response_mlp_agent/puf_response_mlp_agent.keras")

# Create a directory to store the CSV files if it doesn't exist
output_dir = r'models\puf_response_mlp_agent'
os.makedirs(output_dir, exist_ok=True)

for i, layer in enumerate(model.layers):
    if isinstance(layer, layers.Dense):
        weights, biases = layer.get_weights()

        # Save weights to a .npy file (lossless binary format)
        weight_filename = os.path.join(output_dir, f"puf_response_mlp_agent_w{i+1}.npy")
        np.save(weight_filename, weights)
        print(f"Saved weights for {layer.name} to {weight_filename} (shape: {weights.shape})")

        # Save biases to a .npy file
        bias_filename = os.path.join(output_dir, f"puf_response_mlp_agent_b{i+1}.npy")
        np.save(bias_filename, biases)
        print(f"Saved biases for {layer.name} to {bias_filename} (shape: {biases.shape})")

print(f"\nAll weights and biases exported to the '{output_dir}' directory.")

Saved weights for hidden_layer_1 to models\puf_response_mlp_agent\puf_response_mlp_agent_w1.npy (shape: (130, 256))
Saved biases for hidden_layer_1 to models\puf_response_mlp_agent\puf_response_mlp_agent_b1.npy (shape: (256,))
Saved weights for hidden_layer_2 to models\puf_response_mlp_agent\puf_response_mlp_agent_w2.npy (shape: (256, 256))
Saved biases for hidden_layer_2 to models\puf_response_mlp_agent\puf_response_mlp_agent_b2.npy (shape: (256,))
Saved weights for output_layer to models\puf_response_mlp_agent\puf_response_mlp_agent_w3.npy (shape: (256, 128))
Saved biases for output_layer to models\puf_response_mlp_agent\puf_response_mlp_agent_b3.npy (shape: (128,))

All weights and biases exported to the 'models\puf_response_mlp_agent' directory.


In [ ]:
import numpy as np

y_pred_probs = model.predict(X_test)

y_pred = (y_pred_probs > 0.5).astype("int")

per_bit_accuracy = np.mean(y_pred == y_test)
print(f"Per-Bit Accuracy: {per_bit_accuracy * 100:.4f}%")

bit_error_rate = 1 - per_bit_accuracy
print(f"Bit Error Rate (BER): {bit_error_rate * 100:.4f}%")

num_exact_matches = np.sum(np.all(y_pred == y_test, axis=1))
total_test_samples = len(y_test)
print(f"Total Exactly Correct Responses: {num_exact_matches} / {total_test_samples} ({num_exact_matches/total_test_samples * 100:.2f}%)")

print("\n--- Example Predictions vs. True Values ---")
# Displaying the first 5 samples from the test set
for i in range(5):
    print(f"Sample #{i+1}")
    print(f"  Predicted: {''.join(map(str, y_pred[i]))}")
    print(f"  Actual:    {''.join(map(str, y_test[i]))}")
    # Calculate errors for this specific sample
    errors = np.sum(y_pred[i] != y_test[i])
    print(f"  (Errors in this sample: {errors})")
    print("-" * 20)

415/415 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step
Per-Bit Accuracy: 100.0000%
Bit Error Rate (BER): 0.0000%
Total Exactly Correct Responses: 13280 / 13280 (100.00%)

--- Example Predictions vs. True Values ---
Sample #1
  Predicted: 11001011100000100110101101101110010101101100010000101101101010011010100111100000000110001000011100000110000111001101110000100011
  Actual:    11001011100000100110101101101110010101101100010000101101101010011010100111100000000110001000011100000110000111001101110000100011
  (Errors in this sample: 0)
--------------------
Sample #2
  Predicted: 11001010101111000100100101000000000100000101001100100000101100010110000111101000000101000000011100000000000110001100100001100011
  Actual:    11001010101111000100100101000000000100000101001100100000101100010110000111101000000101000000011100000000000110001100100001100011
  (Errors in this sample: 0)
--------------------
Sample #3
  Predicted: 01110010010011000101100000000000001110010110001100100010101100101010101111101000111101

In [ ]:
import numpy as np

ideal_response_bits = y_test[0]

print("--- Running Experiment 1: Zero Vector Input ---")

# Create an input of all zeros (1 sample, 130 features)
zero_input = np.zeros((1, 130))

zero_pred_probs = model.predict(zero_input)
zero_pred_bits = (zero_pred_probs > 0.5).astype(int)[0]

print('Ideal Response bits')
print(ideal_response_bits)
print('Prediction bits')
print(zero_pred_bits)

print('\n')

print("--- Running Experiment 2: Random Vector Input ---")

# 1. Generate a random voltage value (float between 0 and 1)
random_voltage = np.random.rand(1)

# 2. Generate a random temperature value (float between 0 and 1)
random_temperature = np.random.rand(1)

# 3. Generate 128 random bits (integers: 0 or 1)
random_bits = np.random.randint(0, 2, size=128)

# 4. Concatenate all parts into a single 130-element array
final_array = np.concatenate((random_voltage, random_temperature, random_bits))

input_for_model = final_array.reshape(1, -1)

random_bits_probs = model.predict(input_for_model)
random_bits_pred = (random_bits_probs > 0.5).astype(int)[0]

print('Ideal Response bits')
print(ideal_response_bits)
print('Prediction bits')
print(random_bits_pred)


--- Running Experiment 1: Zero Vector Input ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[1 1 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 0 1 0
 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 0 0 1 1 1
 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0 0 1 1 1 0
 0 1 1 0 1 1 1 0 0 0 0 1 0 0 0 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


--- Running Experiment 2: Random Vector Input ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[1 1 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 0 1 0
 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 0 0 1 1 1
 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0 0 1 1 1 0
 0 1 1 0 1 1 1 0 0 0 0 1 0 0 0 1 1]
[1 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 